# Interactive Hyperdrive Tutorial


Agent0 provide a self contained Hyperdrive simulator using Interactive Hyperdrive. It works by managing a local Anvil
instance with an interactive and customizable Hyperdrive deployment, a data collection service, and a dashboard server
showing useful information for analysis.

To start, follow the installation instructions outlined [here](README.md) for interactive hyperdrive.


## Simulating trades using Interactive Hyperdrive

The following code initializes an interactive hyperdrive with (1) a local chain, (2) a deployed hyperdrive pool, and
(3) a funded agent attached to the pool ready to trade.


In [1]:
from fixedpointmath import FixedPoint
from agent0.hyperdrive.interactive import InteractiveHyperdrive, LocalChain

chain = LocalChain()
interactive_hyperdrive = InteractiveHyperdrive(chain)
hyperdrive_agent0 = interactive_hyperdrive.init_agent(base=FixedPoint(100_000))

After initializing, we can use the `hyperdrive_agent` object to simulate trades on the deployed pool.


In [2]:
open_long_event = hyperdrive_agent0.open_long(base=FixedPoint(100))
open_short_event = hyperdrive_agent0.open_short(bonds=FixedPoint(100))
add_liquidity_event = hyperdrive_agent0.add_liquidity(base=FixedPoint(100))

The output of these trades represents the corresponding emitted event from Hyperdrive. For example, the open long event
is shown below. See
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/event_types/index.html) for
documentation on the output event types.


In [3]:
open_long_event

OpenLong(trader='0xc98cf7Def51c730916860Ec1eC2dAA8a638Df767', asset_id=452312848583266388373324160190187140051835877600158453279131187532619741056, maturity_time=1709078400, base_amount=FixedPoint("100.0"), vault_share_price=FixedPoint("1.00000005707762557"), bond_amount=FixedPoint("100.094931446539240406"))

We can use Interactive Hyperdrive to simulate advancing time and closing a position. For example, we can simulate
closing the long after a day.


In [4]:
# Advance time for a day
chain.advance_time(60 * 60 * 24)
close_long_event = hyperdrive_agent0.close_long(
    maturity_time=open_long_event.maturity_time,
    bonds=open_long_event.bond_amount,
)

## Analyzing Hyperdrive

Interactive hyperdrive provides a fully managed data service that collects information from the chain, as
well as providing an interactive dashboard to view information on the pool. Running the dashboard within Interactive
Hyperdrive brings up a webpage that shows you basic information of the trades made on the pool, as well as information
on a specific agent, including the value of the portfolio.


In [5]:
interactive_hyperdrive.run_dashboard()

For customized analysis, Interactive Hyperdrive exposes the underlying data itself. See the `get_*` functions defined in
interactive hyperdrive
[here](https://agent0.readthedocs.io/en/latest/autoapi/agent0/hyperdrive/interactive/interactive_hyperdrive/index.html)
for more information.


## Trading policy

Agent0 introduces an interface to write a policy, which allows a user to define a set of rules for trading on a
deployed hyperdrive pool. In this tutorial, we'll write a policy that simply opens a long if the fixed rate is above a
threshold. This policy can then be attached to interactive hyperdrive to simulate the policy within the simulator, or
executed on a separate chain.

We'll start by defining the class that subclasses from the base policy and define a custom policy
configuration that defines the threshold. We need to overwrite the `action` method in the policy, which defines what
actions the policy takes. In the example below, the agent will simply open a single long for `open_long_amount` when
the fixed rate reaches the `fixed_rate_threshold`, and close the long at maturity.


In [6]:
# Relevant imports
from agent0.hyperdrive.policies import HyperdriveBasePolicy
from dataclasses import dataclass
from ethpy.hyperdrive import HyperdriveReadInterface
from agent0.hyperdrive import HyperdriveWallet
from agent0.hyperdrive.agent import open_long_trade, close_long_trade

In [7]:
class OpenLongPolicy(HyperdriveBasePolicy):
    @dataclass(kw_only=True)
    class Config(HyperdriveBasePolicy.Config):
        fixed_rate_threshold: FixedPoint
        open_long_amount: FixedPoint

    def action(self, interface: HyperdriveReadInterface, wallet: HyperdriveWallet):
        # Defines if the bot is done trading. We expect this bot to run continuously,
        # so this is always false.
        done_trading = False
        # If no longs in wallet, we check our fixed rate threshold and open the long if threshold reached.
        if len(wallet.longs) == 0:
            if interface.calc_fixed_rate() > self.config.fixed_rate_threshold:
                return [open_long_trade(self.config.open_long_amount)], done_trading
        # If there are longs in the wallet, we check for maturity and close them if maturity reached.
        else:
            for maturity_time, long in wallet.longs.items():
                if interface.get_block_timestamp(interface.get_current_block()) >= maturity_time:
                    return [close_long_trade(long.balance, maturity_time)], done_trading
        # We don't do any trades otherwise.
        return [], done_trading

### Running the trading policy in Interactive Hyperdrive

We can test our new policy using interactive hyperdrive before running it on an actual chain. We can do this by
attaching our new policy to a new agent, while passing in a fixed rate threshold of 6%.


In [8]:
policy_agent = interactive_hyperdrive.init_agent(
    base=FixedPoint(1_000_000),
    policy=OpenLongPolicy,
    policy_config=OpenLongPolicy.Config(
        fixed_rate_threshold=FixedPoint(0.06),
        open_long_amount=FixedPoint(100_000),
    ),
)

Lets take a look at the fixed rate after the set of trades we've made. We can see below that the current fixed rate sits
right around 5%.


In [9]:
interactive_hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-02-21 00:35:10,0.049999999999999996
1,21,2024-02-21 00:35:22,0.049999999999999996
2,22,2024-02-21 00:35:34,0.049999999999999996
3,23,2024-02-21 00:35:46,0.049999939182593864
4,24,2024-02-21 00:35:58,0.049999999941555074
5,25,2024-02-21 00:36:10,0.049999999941555074
6,73,2024-02-22 00:36:22,0.049999999941555074
7,74,2024-02-22 00:36:34,0.050000052064673868
8,75,2024-02-22 00:36:46,0.050000052064673868


We can now execute our new agent's policy. Since the fixed rate is below the threshold, we expect the agent to not make
any trades, denoted by the output of `execute_policy_action` being an empty list.


In [10]:
policy_agent.execute_policy_action()

[]

Let's make a trade to push the fixed rate to be 6%.


In [11]:
policy_agent.open_short(bonds=FixedPoint(20_000_000))
interactive_hyperdrive.get_pool_state()[["block_number", "timestamp", "fixed_rate"]]

,block_number,timestamp,fixed_rate
0,20,2024-02-21 00:35:10,0.049999999999999996
1,21,2024-02-21 00:35:22,0.049999999999999996
2,22,2024-02-21 00:35:34,0.049999999999999996
3,23,2024-02-21 00:35:46,0.049999939182593864
4,24,2024-02-21 00:35:58,0.049999999941555074
5,25,2024-02-21 00:36:10,0.049999999941555074
6,73,2024-02-22 00:36:22,0.049999999941555074
7,74,2024-02-22 00:36:34,0.050000052064673868
8,75,2024-02-22 00:36:46,0.050000052064673868
9,76,2024-02-22 00:36:58,0.050000052064673868


Once again, let's execute our policy and view the output. We can see that our policy opened a new long because the
fixed rate is above 6%.


In [12]:
policy_agent.execute_policy_action()

[OpenLong(trader='0xa2548377f006E05AE481d7B3F6eFd6729aC0F5ED', asset_id=452312848583266388373324160190187140051835877600158453279131187532619827456, maturity_time=1709164800, base_amount=FixedPoint("100000.0"), vault_share_price=FixedPoint("1.000137195609470888"), bond_amount=FixedPoint("100119.746891368919969584"))]

### Running policy on existing chain

The policy we implemented here can be used on a separate chain with Hyperdrive deployed. See the [readme](README.md) and the [custom policy example](examples/custom_agent.py) for details on how to execute custom policies on existing chains.
